# alphavec example

This notebook demonstrates running `alphavec.simulate()` on the bundled feather datasets in `tests/testdata/`.

Notes:
- The feather files require `pyarrow`. If you don't have it, install with `pip install pyarrow` in your venv.
- NaNs in prices or funding indicate an asset is not tradable at that period; `simulate()` respects this.
- The run below uses all assets over the full available history.


In [ ]:
from pathlib import Path
from IPython.display import HTML, display
import pandas as pd

from alphavec import simulate, tearsheet


In [ ]:
data_dir = Path("./data")

weights = pd.read_feather(data_dir / "weights.feather")
close_prices = pd.read_feather(data_dir / "close_prices.feather")
order_prices = pd.read_feather(data_dir / "open_prices.feather")
funding_rates = pd.read_feather(data_dir / "funding_rates.feather")

print("weights", weights.shape)
print("close_prices", close_prices.shape)
print("order_prices", order_prices.shape)
print("funding_rates", funding_rates.shape)


In [ ]:
symbols = weights.columns.tolist()
print("assets", len(symbols))
print("start", weights.index.min())
print("end", weights.index.max())

na_summary = {
    "weights_na": int(weights.isna().sum().sum()),
    "close_na": int(close_prices.isna().sum().sum()),
    "order_na": int(order_prices.isna().sum().sum()),
    "funding_na": int(funding_rates.isna().sum().sum()),
}
na_summary


In [ ]:
weights = weights.sort_index()

# Drop leading periods before the strategy has any active (non-NaN) weights.
active_mask = weights.notna().any(axis=1)
weights = weights.loc[active_mask.idxmax() :]

# Align prices/funding to weights; NaNs mean not tradable.
close_all = close_prices.reindex(index=weights.index, columns=weights.columns)
# Use next-period open as the execution price to avoid lookahead.
open_all = order_prices.reindex(index=weights.index, columns=weights.columns)
order_all = open_all.shift(-1)
funding_all = funding_rates.reindex(index=weights.index, columns=weights.columns)

weights.head()


In [ ]:
init_cash = 10_000
benchmark_asset = "BTC"  # set to None to disable alpha/beta

returns, metrics= simulate(
    weights=weights,
    close_prices=close_all,
    order_prices=order_all,
    funding_rates=funding_all,
    benchmark_asset=benchmark_asset,
    order_notional_min=10,
    fee_pct=0.00025,       # 2.5 bps per trade
    slippage_pct=0.001,  # 10 bps per trade
    init_cash=init_cash,
    freq_rule="1D",
    trading_days_year=365,
    risk_free_rate=0.03,
)

html_str = tearsheet(metrics=metrics, returns=returns)
display(HTML(html_str))